In [1]:
import lionagi.libs.ln_func_call as func_call
from lionagi.libs.ln_async import AsyncUtil
from lionagi.core.messages.schema import Instruction, System
from lionagi.core.schema.structure import Structure

from lionagi import Branch
from lionagi.core.mail.schema import BaseMail
from lionagi.core.schema.base_node import BaseRelatableNode
from collections import deque
from typing import Any

from lionagi.core.Agent.Agent import Agent

# Executable class implementation

In [2]:
class Executable_Branch(BaseRelatableNode):

    def __init__(self):
        super().__init__()
        self.branch: Branch = Branch()
        self.pending_ins = {}  ###
        self.pending_outs = deque()  ###
        self.responses = []
        self.execute_stop = False  ###
        self.context = None  ###

    def system_process(self, system: System):
        self.branch.add_message(system=system)

    async def instruction_process(self, instruction: Instruction):
        if self.context:
            instruction.content.update({"context": self.context})
            self.context = None
        result = await self.branch.chat(instruction)
        print("instruction call:", result)
        self.responses.append(result)

    async def agent_process(self, agent):
        context = self.responses
        result = await agent.execute(context)
        print("agent call:", result)
        self.responses.append(result)

    def send(self, recipient_id: str, category: str, package: Any) -> None:
        mail = BaseMail(
            sender_id=self.id_,
            recipient_id=recipient_id,
            category=category,
            package=package,
        )
        self.pending_outs.append(mail)

    async def process(self):
        for key in list(self.pending_ins.keys()):
            while self.pending_ins[key]:
                mail = self.pending_ins[key].popleft()
                if mail.category == "start":  ###
                    start_mail_content = mail.package  ###
                    self.context = start_mail_content["context"]  ###
                    self.send(start_mail_content["structure_id"], "start", "start")  ###
                if mail.category == "node":
                    if isinstance(mail.package, System):
                        self.system_process(mail.package)
                        self.send(mail.sender_id, "node_id", mail.package.id_)
                    elif isinstance(mail.package, Instruction):
                        await self.instruction_process(mail.package)
                        self.send(mail.sender_id, "node_id", mail.package.id_)
                    elif isinstance(mail.package, Agent):
                        await self.agent_process(mail.package)
                        self.send(mail.sender_id, "node_id", mail.package.id_)
                elif mail.category == "end":  ###
                    self.execute_stop = True
                    self.send(mail.sender_id, "end", "end")

    async def execute(self, refresh_time=1):  ###
        while not self.execute_stop:
            await self.process()
            await AsyncUtil.sleep(refresh_time)

# Create Structure

In [3]:
sys_comedian = System(system="As a comedian, you are sarcastically funny")
instruct1 = Instruction(
    instruction="very short joke: a blue whale and a big shark meet at the bar and start dancing"
)
instruct2 = Instruction(instruction="continue the joke: and then they stopped")

In [4]:
struct_comedian = Structure()
struct_comedian.add_node(sys_comedian)
struct_comedian.add_node(instruct1)
struct_comedian.add_node(instruct2)
struct_comedian.add_relationship(sys_comedian, instruct1)
struct_comedian.add_relationship(instruct1, instruct2)

In [5]:
sys_critic = System(
    system="you are a respected commentator, you are artistically logical"
)
instruct3 = Instruction(
    instruction="short comments, what do you think about the first joke?"
)
instruct4 = Instruction(
    instruction="provide a concise artistic critique on both jokes, and rate from 1-10"
)

In [6]:
struct_critic = Structure()
struct_critic.add_node(sys_critic)
struct_critic.add_node(instruct3)
struct_critic.add_node(instruct4)
struct_critic.add_relationship(sys_critic, instruct3)
struct_critic.add_relationship(instruct3, instruct4)

In [7]:
## output_parser_func parameter: agent self


def critic_output_parser(agent):
    return agent.executable.responses

In [8]:
critic = Agent(
    structure=struct_critic,
    executable_class=Executable_Branch,
    output_parser=critic_output_parser,
)

In [9]:
struct_comedian.add_node(critic)
struct_comedian.add_relationship(instruct2, critic)

In [10]:
instruct5 = Instruction(
    instruction="your jokes were evaluated by a critic, does it make sense to you? why?"
)
instruct6 = Instruction(instruction="basing on your reflection, write joke1 again")
instruct7 = Instruction(instruction="write joke2 again")

In [11]:
struct_comedian.add_node(instruct5)
struct_comedian.add_node(instruct6)
struct_comedian.add_node(instruct7)
struct_comedian.add_relationship(critic, instruct5)
struct_comedian.add_relationship(instruct5, instruct6)
struct_comedian.add_relationship(instruct6, instruct7)

In [12]:
## output_parser_func parameter: agent self


def comedian_output_parse(agent):
    return agent.executable.responses

In [13]:
comedian = Agent(
    structure=struct_comedian,
    executable_class=Executable_Branch,
    output_parser=comedian_output_parse,
)

# Execute

In [14]:
result = await comedian.execute()

instruction call: Why did the blue whale and the big shark start dancing at the bar? Because they heard it was a fin-tastic place to have a whale of a time!
instruction call: And then they stopped dancing. Why? Because they realized the bar didn't serve fish and chips. Talk about a party pooper, huh?
instruction call: The first joke charmingly blends oceanic wordplay and a dash of whimsy to create a delightful punchline. It cleverly plays on the expectations of its audience with a light-hearted twist.
instruction call: Both jokes exhibit a playful use of language, intertwining aquatic themes with humor. The first joke shines with its pun-laden setup, leading to a whimsically amusing punchline that encapsulates a 'fin-tastic' sense of joy, meriting a 7/10 for its clever wordplay and light-hearted appeal. The second joke, while maintaining the aquatic theme, pivots towards an anticlimactic realization, using the absence of fish and chips as a humorous letdown. It cleverly builds on the s